In [2]:
# Libraries:
library(dplyr)

# Function 1:
function1 <- function(m,n,x,H,ref_samp) {
  N=m+n
  
  test <- data.frame(round(rnorm(n,mean = x),2),1)
  colnames(test) <- c("dat","z_k") 
  
  single <- rbind(ref_samp,test)
  single <- single[order(single$dat),]
  single %>% mutate(
    k = 1:nrow(single),
    t_1 = k*z_k,
    t_2 = abs(k - .5*(N+1))*z_k) -> single
  
  metrics <- data.frame(sum(single$t_1))
  colnames(metrics) <- "T_1"
  metrics$T_2 <- sum(single$t_2)
  metrics$sigma_1_sqrd <- (1/12)*m*n*(N+1)
  metrics$mu_1 <- .5*n*(N+1)
  
  if((N %% 2) == 0) {
    # even
    metrics$simga_2_sqrd <- (1/48)*m*n*((N^2-4)/(N-1))
    metrics$mu_2 <- (n*N)/4
  } else {
    # odd
    metrics$sigma_2_sqrd <- (1/48)*(m*n*(N+1)*(N^2+3))/(N^2)
    metrics$mu_2 <- (n*(N^2-1))/(4*N)
  }
  
  metrics %>% mutate(
    S_1 = (T_1 - mu_1)/sqrt(sigma_1_sqrd),
    S_2 = (T_2 - mu_2)/sqrt(sigma_2_sqrd),
    Lapage = S_1^2 + S_2^2) -> metrics
  
  if (metrics$Lapage < H) {
    metrics$process <- "IC"
  } else {
    metrics$process <- "OOC"
  }
  return(metrics)
  
}

In [3]:
# Function 2:
function2 <- function(m,n,x,H) {
  ref <- data.frame(round(rnorm(m,mean = 0.5),2),0)
  colnames(ref) <- c("dat","z_k")
  
  set <- replicate(1000, function1(m,n,x,H,ref), simplify=FALSE)
  set <- do.call(rbind,set)
  #print(table(set$process))
  return(set)
}

In [4]:
# Function 3:
function3 <- function(m,n,x,H,w) {
  full_set <- replicate(w,function2(m,n,x,H),simplify=FALSE)
  return(full_set)
}

In [6]:
# Inputs for function 3:
m=30
n=11
H = 9.24
w=30
x=0.5
test1 <- function3(m,n,x,H,w)

In [7]:
# Calculate ARL
total_RL = 0
n_errors = 0
total_RL.2 = 0
n_errors.2 = 0
for (i in 1:w) {
  set <- test1[[i]]
  RL_i <- as.numeric(which(set$process == "OOC", arr.ind=TRUE))
  
  if (length(RL_i)==0) {
    n_errors <- n_errors + 1
    #print(i)
    
  } else{
    total_RL <- total_RL + mean(RL_i)
    
    
  }
  
  # 2/3 rule
  for (k in 1:(nrow(set)-2)) {
    vec <- set[k:(k+2),"process"]
    #se <- names(table(vec))[as.vector(table(vec))==max(table(vec))]
    #print(se)
    
    set$process.2out3[k] <- names(table(vec))[as.vector(table(vec))==max(table(vec))]
    #print(set$process.2out3[1:20])
  }
  RL_i.2 <- as.numeric(which(set$process.2out3 == "OOC", arr.ind=TRUE))
  #print(RL_i.2)
  #print(length(RL_i.2))
  if (length(RL_i.2)==0) {
    n_errors.2 <- n_errors.2 + 1
    #print(n_errors.2)
    
    print(paste("2:",i))
  } else{
    total_RL.2 <- total_RL.2 + mean(RL_i.2)
    #print(total_RL.2)
  }
}
print(total_RL)
print(n_errors)
print(total_RL.2)
print(n_errors.2)
ARL = (total_RL)/(w-n_errors)
ARL.2 = (total_RL.2)/(w-n_errors.2)

[1] "2: 1"
[1] "2: 2"
[1] "2: 3"
[1] "2: 4"
[1] "2: 5"
[1] "2: 6"
[1] "2: 7"
[1] "2: 8"
[1] "2: 12"
[1] "2: 13"
[1] "2: 15"
[1] "2: 16"
[1] "2: 17"
[1] "2: 18"
[1] "2: 19"
[1] "2: 20"
[1] "2: 21"
[1] "2: 22"
[1] "2: 23"
[1] "2: 24"
[1] "2: 25"
[1] "2: 26"
[1] "2: 27"
[1] "2: 28"
[1] "2: 29"
[1] "2: 30"
[1] 13723.18
[1] 2
[1] 1618.167
[1] 26


In [8]:
print(ARL)
print(ARL.2)

[1] 490.1134
[1] 404.5417


In [15]:
# Inputs for function 3:
m=30
n=11
H =8
w=30
x=1
test1 <- function3(m,n,x,H,w)

In [16]:
# Calculate ARL
total_RL = 0
n_errors = 0
total_RL.2 = 0
n_errors.2 = 0
for (i in 1:w) {
  set <- test1[[i]]
  RL_i <- as.numeric(which(set$process == "OOC", arr.ind=TRUE))
  
  if (length(RL_i)==0) {
    n_errors <- n_errors + 1
    #print(i)
    
  } else{
    total_RL <- total_RL + mean(RL_i)
    
    
  }
  
  # 2/3 rule
  for (k in 1:(nrow(set)-2)) {
    vec <- set[k:(k+2),"process"]
    #se <- names(table(vec))[as.vector(table(vec))==max(table(vec))]
    #print(se)
    
    set$process.2out3[k] <- names(table(vec))[as.vector(table(vec))==max(table(vec))]
    #print(set$process.2out3[1:20])
  }
  RL_i.2 <- as.numeric(which(set$process.2out3 == "OOC", arr.ind=TRUE))
  #print(RL_i.2)
  #print(length(RL_i.2))
  if (length(RL_i.2)==0) {
    n_errors.2 <- n_errors.2 + 1
    #print(n_errors.2)
    
    print(paste("2:",i))
  } else{
    total_RL.2 <- total_RL.2 + mean(RL_i.2)
    #print(total_RL.2)
  }
}
print(total_RL)
print(n_errors)
print(total_RL.2)
print(n_errors.2)
ARL = (total_RL)/(w-n_errors)
ARL.2 = (total_RL.2)/(w-n_errors.2)
print(ARL)
print(ARL.2)

[1] "2: 6"
[1] "2: 9"
[1] "2: 10"
[1] "2: 12"
[1] "2: 13"
[1] "2: 14"
[1] "2: 23"
[1] "2: 29"
[1] "2: 30"
[1] 14363.49
[1] 0
[1] 9663.387
[1] 9
[1] 478.7829
[1] 460.1613


In [17]:
print(ARL)
print(ARL.2)

[1] 478.7829
[1] 460.1613


In [24]:
# Inputs for function 3:
m=30
n=11
H =10
w=30
x=1.5
test1 <- function3(m,n,x,H,w)

In [25]:
# Calculate ARL
total_RL = 0
n_errors = 0
total_RL.2 = 0
n_errors.2 = 0
for (i in 1:w) {
  set <- test1[[i]]
  RL_i <- as.numeric(which(set$process == "OOC", arr.ind=TRUE))
  
  if (length(RL_i)==0) {
    n_errors <- n_errors + 1
    #print(i)
    
  } else{
    total_RL <- total_RL + mean(RL_i)
    
    
  }
  
  # 2/3 rule
  for (k in 1:(nrow(set)-2)) {
    vec <- set[k:(k+2),"process"]
    #se <- names(table(vec))[as.vector(table(vec))==max(table(vec))]
    #print(se)
    
    set$process.2out3[k] <- names(table(vec))[as.vector(table(vec))==max(table(vec))]
    #print(set$process.2out3[1:20])
  }
  RL_i.2 <- as.numeric(which(set$process.2out3 == "OOC", arr.ind=TRUE))
  #print(RL_i.2)
  #print(length(RL_i.2))
  if (length(RL_i.2)==0) {
    n_errors.2 <- n_errors.2 + 1
    #print(n_errors.2)
    
    print(paste("2:",i))
  } else{
    total_RL.2 <- total_RL.2 + mean(RL_i.2)
    #print(total_RL.2)
  }
}
print(total_RL)
print(n_errors)
print(total_RL.2)
print(n_errors.2)
ARL = (total_RL)/(w-n_errors)
ARL.2 = (total_RL.2)/(w-n_errors.2)
print(ARL)
print(ARL.2)

[1] 14924.72
[1] 0
[1] 15137.09
[1] 0
[1] 497.4906
[1] 504.5698
